In [31]:
import schedule
import sys
import time
import requests
import pandas as pd
from datetime import datetime, timedelta, date
import pyodbc
from sqlalchemy import create_engine
import mysql.connector

In [32]:
db_config = {
    'host': 'Localhost',
    'user': 'root',
    'password': '8november2002',
    'database': 'bhavDB',
}
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()
# engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")

In [33]:
import pandas as pd
def greenSPY(df_today, df_prev, df_prev2):
    symbols = df_today['SYMBOL'].unique().tolist()

    for symbol in symbols:
        today = df_today[df_today['SYMBOL'] == symbol]
        #print(today)
        prev_day = df_prev[df_prev['SYMBOL'] == symbol]
        #print(prev_day)
        prev_day2 = df_prev2[df_prev2['SYMBOL'] == symbol]
        #print(prev_day2)

        # Check if any of the filtered DataFrames is empty
        if today.empty or prev_day.empty or prev_day2.empty:
            continue

        today = today.iloc[0]
        second_day = prev_day.iloc[0]
        first_day = prev_day2.iloc[0]

        # Apply condition to find colors of all three days candles
        second_day_color = 'red' if second_day['OPEN_PRICE'] > second_day['CLOSE_PRICE'] else 'green'
        first_day_color = 'green' if first_day['OPEN_PRICE'] < first_day['CLOSE_PRICE'] else 'red'
        today_color = 'green' if today['OPEN_PRICE'] < today['CLOSE_PRICE'] else 'red'

        # Condition check
        condition_met = (
            first_day_color == 'red' and
            second_day_color == 'green' and
            today_color == 'green' and
            second_day['HIGH_PRICE'] < first_day['OPEN_PRICE'] and
            second_day['LOW_PRICE'] > first_day['CLOSE_PRICE'] and
            today['CLOSE_PRICE'] > first_day['HIGH_PRICE']
        )
        # print(f"For Symbol: {symbol, condition_met}")
        # Check if the condition is met and proceed
        if condition_met:
            # entry_price = today['OPEN_PRICE']
            stop_loss = first_day['LOW_PRICE']
            # target_price = entry_price * 1.1

            print(f"For Symbol: {symbol}")
            # print(f"Buy Signal - Entry Price:  Tomorrow Open Price")
            print(f"Stop Loss: {stop_loss}")
            # print(f"Target Price: entry_price * 1.1")
            print()

In [34]:
data=[]
date="28012024"
current_date=datetime.strptime(date,"%d%m%Y") #25-01-2024
counter=0
while(True):
    #Check response and append data
    query=f"SELECT * FROM StockData WHERE DATE1 = '{current_date}'"
    cursor.execute(query)
    result= cursor.fetchall()
    if(len(result)!=0):
        data.append(result)
        counter+=1
        print(f"Open {current_date}")
    else:
        print(f"Close {current_date}")
    current_date-=timedelta(days=1)
    #Break Loop
    if(counter==3):
        break
print(data)

Close 2024-01-28 00:00:00
Close 2024-01-27 00:00:00
Close 2024-01-26 00:00:00
Open 2024-01-25 00:00:00
Open 2024-01-24 00:00:00
Open 2024-01-23 00:00:00
[[('20MICRONS', 'EQ', datetime.date(2024, 1, 25), 166.9, 169.0, 177.7, 166.05, 176.0, 175.6, 173.14, 242415, 419.71, 12301, '111016', '45.80'), ('21STCENMGM', 'BE', datetime.date(2024, 1, 25), 39.8, 40.55, 40.55, 40.55, 40.55, 40.55, 40.55, 4838, 1.96, 19, '-', '-'), ('360ONE', 'EQ', datetime.date(2024, 1, 25), 617.0, 617.0, 637.4, 614.95, 631.35, 630.2, 627.31, 244105, 1531.3, 10786, '172730', '70.76'), ('3IINFOLTD', 'EQ', datetime.date(2024, 1, 25), 57.2, 58.0, 58.45, 56.95, 58.0, 57.75, 57.61, 1912961, 1102.02, 7453, '664612', '34.74'), ('3MINDIA', 'EQ', datetime.date(2024, 1, 25), 34470.9, 34166.0, 34756.1, 33950.0, 34000.1, 34048.2, 34233.7, 2771, 948.62, 1417, '1443', '52.08'), ('3PLAND', 'EQ', datetime.date(2024, 1, 25), 36.5, 36.1, 37.35, 34.7, 35.0, 34.95, 35.76, 55178, 19.73, 493, '42634', '77.27'), ('5PAISA', 'EQ', datetime.

In [35]:
column_names = [desc[0] for desc in cursor.description]
print(column_names)

['SYMBOL', 'SERIES', 'DATE1', 'PREV_CLOSE', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'LAST_PRICE', 'CLOSE_PRICE', 'AVG_PRICE', 'TTL_TRD_QNTY', 'TURNOVER_LACS', 'NO_OF_TRADES', 'DELIV_QTY', 'DELIV_PER']


In [36]:
df_today = pd.DataFrame(data[0], columns=column_names)
df_prev = pd.DataFrame(data[1], columns=column_names)
df_prev2 = pd.DataFrame(data[2], columns=column_names)

In [37]:
greenSPY(df_today, df_prev, df_prev2)

For Symbol: BBTC
Stop Loss: 1528.4

For Symbol: DHUNINV
Stop Loss: 1133.0

For Symbol: MIRCELECTR
Stop Loss: 15.85

For Symbol: NMDC
Stop Loss: 201.5

For Symbol: SHRIPISTON
Stop Loss: 1506.0

For Symbol: UGARSUGAR
Stop Loss: 76.0



In [38]:
connection.commit()
cursor.close()
connection.close()